### Get data from .mat format to dict 
and save as python-friendly format

nina 2020.3.4

In [1]:
import scipy.io
matfile_path = '../S2_data/data.mat'
mat = scipy.io.loadmat(matfile_path)

In [2]:
mdata = mat['data']

In [3]:
mdata.dtype.names
# mdata['AB']

('AB',
 'AG',
 'VB',
 'VG',
 'AVB',
 'AVG',
 'AVFus',
 'dim_order',
 'response_order',
 'nsubjects')

In [4]:
print(mdata['AB'][0,0].shape)
print(mdata['AB'][0,0]['counts'][0,0].shape)

(1, 1)
(16, 3, 3)


In [5]:
ndata = {n: mdata[n][0,0] for n in mdata.dtype.names}
dict_data = {}
for each in ndata:
    print('processing {}'.format(each))
    each_d = ndata[each]
    if len(each_d.dtype) == 0:
        dict_data[each] = each_d
        continue
    each_d_n = {n: each_d[n][0,0] for n in each_d.dtype.names}
    
    dict_tmp={}
    for each_2 in each_d_n:
        print('processing {}'.format(each_2))
        each_dd = each_d_n[each_2]
        if len(each_dd.dtype) == 0:
            dict_tmp[each_2] = each_dd
            continue
        
        each_d_nn = {n: each_dd[n][0,0] for n in each_dd.dtype.names}
        dict_tmp[each_2] = each_d_nn
    
    dict_data[each] = dict_tmp

    
print('pre-processing done')

processing AB
processing counts
processing props
processing snr
processing AG
processing counts
processing props
processing snr
processing VB
processing counts
processing props
processing snr
processing VG
processing counts
processing props
processing snr
processing AVB
processing synch
processing asynch
processing AVG
processing synch
processing asynch
processing AVFus
processing synch
processing asynch
processing dim_order
processing response_order
processing nsubjects
pre-processing done


In [6]:
dict_data.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])

In [7]:
dict_data['AVG'].keys()

dict_keys(['synch', 'asynch'])

In [12]:
dict_data['AVB']['synch'].keys()

dict_keys(['counts', 'snr'])

##### problem: 'AVB' do not have 'props'

In [16]:
dict_data['AVB']['synch']['props'] = dict_data['AVB']['synch']['counts']/25

In [17]:
dict_data['AVB']['synch'].keys()

dict_keys(['counts', 'snr', 'props'])

In [18]:
dict_data['AVB']['synch']['props'] 

array([[[0.  , 0.  , 1.  ],
        [0.04, 0.  , 0.96],
        [0.  , 0.  , 1.  ],
        [0.04, 0.08, 0.88],
        [0.  , 0.  , 1.  ]],

       [[0.16, 0.  , 0.84],
        [0.44, 0.  , 0.56],
        [0.32, 0.  , 0.68],
        [0.08, 0.  , 0.92],
        [0.08, 0.  , 0.92]],

       [[0.08, 0.  , 0.92],
        [0.16, 0.  , 0.84],
        [0.4 , 0.04, 0.56],
        [0.08, 0.08, 0.84],
        [0.16, 0.08, 0.76]],

       [[0.  , 0.  , 1.  ],
        [0.16, 0.  , 0.84],
        [0.12, 0.  , 0.88],
        [0.04, 0.  , 0.96],
        [0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 1.  ],
        [0.08, 0.04, 0.88],
        [0.36, 0.24, 0.4 ],
        [0.04, 0.  , 0.96],
        [0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 1.  ],
        [0.  , 0.  , 1.  ],
        [0.  , 0.  , 1.  ],
        [0.  , 0.  , 1.  ],
        [0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 1.  ],
        [0.08, 0.  , 0.92],
        [0.2 , 0.  , 0.8 ],
        [0.  , 0.  , 1.  ],
        [0.  , 0.  , 1.  ]],

      

In [19]:
dict_data['AVFus']['asynch']['props']

array([[[0.24, 0.  , 0.76],
        [0.28, 0.  , 0.72],
        [0.16, 0.  , 0.84],
        [0.16, 0.08, 0.76],
        [0.2 , 0.12, 0.68]],

       [[0.56, 0.  , 0.44],
        [0.56, 0.04, 0.4 ],
        [0.52, 0.  , 0.48],
        [0.52, 0.04, 0.44],
        [0.76, 0.  , 0.24]],

       [[0.28, 0.16, 0.56],
        [0.28, 0.16, 0.56],
        [0.52, 0.04, 0.44],
        [0.2 , 0.4 , 0.4 ],
        [0.32, 0.56, 0.12]],

       [[0.6 , 0.  , 0.4 ],
        [0.52, 0.  , 0.48],
        [0.32, 0.  , 0.68],
        [0.88, 0.  , 0.12],
        [0.96, 0.  , 0.04]],

       [[0.32, 0.64, 0.04],
        [0.4 , 0.36, 0.24],
        [0.28, 0.28, 0.44],
        [0.12, 0.76, 0.12],
        [0.24, 0.76, 0.  ]],

       [[0.  , 0.  , 1.  ],
        [0.08, 0.  , 0.92],
        [0.  , 0.  , 1.  ],
        [0.08, 0.  , 0.92],
        [0.28, 0.  , 0.72]],

       [[0.32, 0.04, 0.64],
        [0.24, 0.  , 0.76],
        [0.16, 0.  , 0.84],
        [0.36, 0.12, 0.52],
        [0.64, 0.08, 0.28]],

      

##### Store the data into python-friendly format

1. store the dict data into pickle format.

In [20]:
import pickle

pkl_path = '../S2_data/data.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump(dict_data, f, pickle.HIGHEST_PROTOCOL)


try to read this file again:

In [21]:
with open(pkl_path, 'rb') as f:
    data_newv = pickle.load(f)

In [22]:
data_newv.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])

2. store it into npy format

In [23]:
import numpy as np
npy_filepath  = '../S2_data/data.npy'
np.save(npy_filepath, dict_data) 

In [24]:
# fix the 'Object arrays cannot be loaded when allow_pickle=False' problem
# https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # call load_data with allow_pickle implicitly set to true
read_dict = np.load(npy_filepath).item()

# restore np.load for future normal usage
np.load = np_load_old

In [25]:
read_dict.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])

#### problems that remains
- AVB do not has 'props'